&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&ensp;
[Home Page](Start_Here.ipynb)
    
    
[Previous Notebook](Getting_started_with_Deepstream_Pipeline.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;
[1](Introduction_to_Deepstream_and_Gstreamer.ipynb)
[2](Getting_started_with_Deepstream_Pipeline.ipynb)
[3]
[4](Multi-stream_pipeline.ipynb)
[5](Multi-stream_Multi_DNN.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[Next Notebook](Multi-stream_pipeline.ipynb)

# Introduction to Multi-DNN pipeline 

In the previous notebook we learnt on how to make a simple DeepStream pipeline for object detection. In this notebook we take the idea forward and learn to build a  multi-class object detection,tracking and attribute classification pipeline


**Contents of this Notebook :**

- [Changes in configuration for a Multi-DNN pipeline](#Changes-in-configuration)
- [Nvtracker](#Nvtracker)
- [Building the pipeline](#Building-the-Pipeline) 

![test2](images/test2.png)

We can explore the architecture diagram of the application. Here, we have 3 additional models that identify car color, make and type respectively. Plugging in additional models is like adding the original classifier, however there are configuration considerations to take care of.A new idea that we will be using in this notebook is the nvtracker plugin.  

### Changes in configuration

Because these secondary classifiers are only intended to execute on objects that we believe are vehicles, we will need to add new configuration parameters to generate this behavior.Two new parameters, `operate-on-gie-id` and `operate-on-class-ids` will let us control this behavior.

The first, `operate-on-gie-id`, lets us configure a classifier to only execute on objects from a
different classifier. In this case, we will configure the secondary classifier to only execute on
objects detected by the primary classifier. The second, `operate-on-class-ids`, lets us configure a
classifier to only execute on objects of a specific class. By combining these two, our secondary
classifiers will be configured to only evaluate the make, model, and color of objects classified as
cars by our primary model.



# Nvtracker

The plugin accepts NV12/RGBA data from the upstream component and scales (converts) the input buffer to a Luma buffer with a specific tracker width and height. (Tracker width and height must be specified in the configuration file's [tracker] section.) 

The low-level library uses a CPU based implementation of the Kanade Lucas Tomasi (KLT)  tracker algorithm. The plugin also supports the Intersection of Union (IoU) tracker algorithm, which uses the intersection of the detector’s bounding boxes across frames to determine the object's unique ID.

![nvtracker](images/nvtracker.png)

**DeepStream 5.1** included three low-level tracker libraries:
- `libnvds_mot_iou.so`
- `libnvds_mot_klt.so`
- `libnvds_nvdcf.so`

All of these three low-level tracker libraries are deprecated in **DeepStream 6.0.1**, and a unified low-level tracker library is introduced, named:
- `libnvds_nvmultiobjecttracker.so`

Therefore, the tracker configuration file depending on your tracker choice of `IOU`, `NvDCF (w/ perf-oriented setting)`, `NvDCF (w/ accuracy-oriented setting)`, or `DeepSORT`  


```bash
#ll-lib-file=/opt/nvidia/deepstream/deepstream/lib/libnvds_mot_iou.so
ll-lib-file=/opt/nvidia/deepstream/deepstream/lib/libnvds_mot_klt.so
#ll-lib-file=/opt/nvidia/deepstream/deepstream/lib/libnvds_nvdcf.so
ll-config-file=tracker_config.yml
```
would be replaced with:

```bash
ll-lib-file=/opt/nvidia/deepstream/deepstream/lib/libnvds_nvmultiobjecttracker.so
#ll-config-file=config_tracker_IOU.yml
ll-config-file=config_tracker_NvDCF_perf.yml
#ll-config-file=config_tracker_NvDCF_accuracy.yml
#ll-config-file=config_tracker_DeepSORT.yml

```

The tracker component updates the object’s metadata with a tracker-id. After this component, we
add three cascaded secondary neural network classifiers. These classifiers work on the objects
detected as “vehicles or cars”. The first classifier classifies the car color. The second classifier
classifies the car make, and the third classifier classifies car type (e.g. coupe, sedan, etc.). Each
classifier, after inference on a car object, will append the metadata to their results. Then, the
application, using a callback function, can access the metadata to understand and analyze the
attributes of the objects.


### Building the Pipeline

Let us now build the pipeline in a similar fashion as describe in the previous notebook.

In [ ]:
# Import Required Libraries 
import sys
sys.path.append('../source_code')
import gi
import time
import configparser
gi.require_version('Gst', '1.0')
from gi.repository import GObject, Gst
from common.bus_call import bus_call
import pyds

# Defining the Class Labels
PGIE_CLASS_ID_VEHICLE = 0
PGIE_CLASS_ID_BICYCLE = 1
PGIE_CLASS_ID_PERSON = 2
PGIE_CLASS_ID_ROADSIGN = 3

# Defining the input output video file 
INPUT_VIDEO_NAME  = '/opt/nvidia/deepstream/deepstream-6.0/samples/streams/sample_720p.h264'
OUTPUT_VIDEO_NAME = "../source_code/N2/ds_out.mp4"

We define a function `make_elm_or_print_err()` to create our elements and report any errors if the creation fails.

Elements are created using the `Gst.ElementFactory.make()` function as part of Gstreamer library.

In [ ]:
## Make Element or Print Error and any other detail
def make_elm_or_print_err(factoryname, name, printedname, detail=""):
  print("Creating", printedname)
  elm = Gst.ElementFactory.make(factoryname, name)
  if not elm:
     sys.stderr.write("Unable to create " + printedname + " \n")
  if detail:
     sys.stderr.write(detail)
  return elm

#### Initialise GStreamer and Create an Empty Pipeline

In [ ]:
# Standard GStreamer initialization
GObject.threads_init()
Gst.init(None)

# Create gstreamer elements
# Create Pipeline element that will form a connection of other elements
print("Creating Pipeline \n ")
pipeline = Gst.Pipeline()

if not pipeline:
    sys.stderr.write(" Unable to create Pipeline \n")

#### Create Elements that are required for our pipeline 

In [ ]:
########### Create Elements required for the Pipeline ########### 
# Source element for reading from the file
source = make_elm_or_print_err("filesrc", "file-source","Source")
# Since the data format in the input file is elementary h264 stream we need a h264parser
h264parser = make_elm_or_print_err("h264parse", "h264-parser","h264 parse")
# Use nvdec_h264 for hardware accelerated decode on GPU
decoder = make_elm_or_print_err("nvv4l2decoder", "nvv4l2-decoder","Nvv4l2 Decoder")
# Create nvstreammux instance to form batches from one or more sources.
streammux = make_elm_or_print_err("nvstreammux", "Stream-muxer",'NvStreamMux')
# Use nvinfer to run inferencing on decoder's output, behaviour of inferencing is set through config file
pgie = make_elm_or_print_err("nvinfer", "primary-inference" ,"pgie")
# Use nvtracker to give objects unique-ids
tracker = make_elm_or_print_err("nvtracker", "tracker",'tracker')
# Seconday inference for Finding Car Color
sgie1 = make_elm_or_print_err("nvinfer", "secondary1-nvinference-engine",'sgie1')
# Seconday inference for Finding Car Make
sgie2 = make_elm_or_print_err("nvinfer", "secondary2-nvinference-engine",'sgie2')
# Seconday inference for Finding Car Type
sgie3 = make_elm_or_print_err("nvinfer", "secondary3-nvinference-engine",'sgie3')
# Use convertor to convert from NV12 to RGBA as required by nvosd
nvvidconv = make_elm_or_print_err("nvvideoconvert", "convertor","nvvidconv")
# Create OSD to draw on the converted RGBA buffer
nvosd = make_elm_or_print_err("nvdsosd", "onscreendisplay","nvosd")
# Finally encode and save the osd output
queue = make_elm_or_print_err("queue", "queue", "Queue")
# Use convertor to convert from NV12 to RGBA as required by nvosd
nvvidconv2 = make_elm_or_print_err("nvvideoconvert", "convertor2","nvvidconv2")
# Place an encoder instead of OSD to save as video file
encoder = make_elm_or_print_err("avenc_mpeg4", "encoder", "Encoder")
# Parse output from Encoder 
codeparser = make_elm_or_print_err("mpeg4videoparse", "mpeg4-parser", 'Code Parser')
# Create a container
container = make_elm_or_print_err("qtmux", "qtmux", "Container")
# Create Sink for storing the output 
sink = make_elm_or_print_err("filesink", "filesink", "Sink")

Now that we have created the elements ,we can now set various properties for out pipeline at this point. 

For the sgie1 , sgie2 and sgie3 , we use `operate-on-gie-id` and `operate-on-class-ids`, we configured the pipeline to only evaluate the make, model, and color of objects classified as cars by our primary model.

You can access the configuration files here : [pgie](../source_code/N2/dstest2_pgie_config.txt) , [sgie1](../source_code/N2/dstest2_sgie1_config.txt) , [sgie2](../source_code/N2/dstest2_sgie2_config.txt) , [sgie3](../source_code/N2/dstest2_sgie3_config.txt)

In [ ]:
############ Set properties for the Elements ############
print("Playing file %s",INPUT_VIDEO_NAME)
# Set Input File Name 
source.set_property('location', INPUT_VIDEO_NAME)
# Set Input Width , Height and Batch Size 
streammux.set_property('width', 1920)
streammux.set_property('height', 1080)
streammux.set_property('batch-size', 1)
# Set Timeout in microseconds to wait after the first buffer is  
# available to push the batch even if a complete batch is not formed.
streammux.set_property('batched-push-timeout', 4000000)
# Set Congifuration file for nvinfer 
pgie.set_property('config-file-path', "../source_code/N2/dstest2_pgie_config.txt")
sgie1.set_property('config-file-path', "../source_code/N2/dstest2_sgie1_config.txt")
sgie2.set_property('config-file-path', "../source_code/N2/dstest2_sgie2_config.txt")
sgie3.set_property('config-file-path', "../source_code/N2/dstest2_sgie3_config.txt")
#Set properties of tracker from tracker_config
config = configparser.ConfigParser()
config.read('../source_code/N2/dstest2_tracker_config.txt')
config.sections()
for key in config['tracker']:
    if key == 'tracker-width' :
        tracker_width = config.getint('tracker', key)
        tracker.set_property('tracker-width', tracker_width)
    if key == 'tracker-height' :
        tracker_height = config.getint('tracker', key)
        tracker.set_property('tracker-height', tracker_height)
    if key == 'gpu-id' :
        tracker_gpu_id = config.getint('tracker', key)
        tracker.set_property('gpu_id', tracker_gpu_id)
    if key == 'll-lib-file' :
        tracker_ll_lib_file = config.get('tracker', key)
        tracker.set_property('ll-lib-file', tracker_ll_lib_file)
    if key == 'll-config-file' :
        tracker_ll_config_file = config.get('tracker', key)
        tracker.set_property('ll-config-file', tracker_ll_config_file)
    if key == 'enable-batch-process' :
        tracker_enable_batch_process = config.getint('tracker', key)
        tracker.set_property('enable_batch_process', tracker_enable_batch_process)

# Set Encoder bitrate for output video
encoder.set_property("bitrate", 2000000)
# Set Output file name and disable sync and async
sink.set_property("location", OUTPUT_VIDEO_NAME)
sink.set_property("sync", 0)
sink.set_property("async", 0)

We now link all the elements in the order we prefer and create Gstreamer bus to feed all messages through it. 

In [ ]:
########## Add and Link ELements in the Pipeline ########## 

print("Adding elements to Pipeline \n")
pipeline.add(source)
pipeline.add(h264parser)
pipeline.add(decoder)
pipeline.add(streammux)
pipeline.add(pgie)
pipeline.add(tracker)
pipeline.add(sgie1)
pipeline.add(sgie2)
pipeline.add(sgie3)
pipeline.add(nvvidconv)
pipeline.add(nvosd)
pipeline.add(queue)
pipeline.add(nvvidconv2)
pipeline.add(encoder)
pipeline.add(codeparser)
pipeline.add(container)
pipeline.add(sink)

# We now  link the elements together 
# file-source -> h264-parser -> nvh264-decoder -> nvinfer -> nvtracker -> 
# nvinfer_secondary1 -> nvinfer_secondary2 -> nvinfer_secondary2 -> nvvidconv ->
# queue -> nvvidconv2 -> encoder -> parser -> container -> sink -> output-file
print("Linking elements in the Pipeline \n")

source.link(h264parser)
h264parser.link(decoder)

sinkpad = streammux.get_request_pad("sink_0")
if not sinkpad:
    sys.stderr.write(" Unable to get the sink pad of streammux \n")
    
srcpad = decoder.get_static_pad("src")
if not srcpad:
    sys.stderr.write(" Unable to get source pad of decoder \n")
    
srcpad.link(sinkpad)
streammux.link(pgie)
pgie.link(tracker)
tracker.link(sgie1)
sgie1.link(sgie2)
sgie2.link(sgie3)
sgie3.link(nvvidconv)
nvvidconv.link(nvosd)
nvosd.link(queue)
queue.link(nvvidconv2)
nvvidconv2.link(encoder)
encoder.link(codeparser)
codeparser.link(container)
container.link(sink)

In [ ]:
# create and event loop and feed gstreamer bus mesages to it
loop = GObject.MainLoop()

bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect ("message", bus_call, loop)

Our pipeline now carries the metadata forward but we have not done anything with it until now, but as mentoioned in the above pipeline diagram , we will now create a callback function to write relevant data on the frame once called and create a sink pad in the nvosd element and link it to the callback function. 

This callback function is the same as used in the previous notebook.

In [ ]:
############## Working with the Metadata ################

def osd_sink_pad_buffer_probe(pad,info,u_data):
    
    #Intiallizing object counter with 0.
    obj_counter = {
        PGIE_CLASS_ID_VEHICLE:0,
        PGIE_CLASS_ID_PERSON:0,
        PGIE_CLASS_ID_BICYCLE:0,
        PGIE_CLASS_ID_ROADSIGN:0
    }
    # Set frame_number & rectangles to draw as 0 
    frame_number=0
    num_rects=0
    
    gst_buffer = info.get_buffer()
    if not gst_buffer:
        print("Unable to get GstBuffer ")
        return

    # Retrieve batch metadata from the gst_buffer
    # Note that pyds.gst_buffer_get_nvds_batch_meta() expects the
    # C address of gst_buffer as input, which is obtained with hash(gst_buffer)
    batch_meta = pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    l_frame = batch_meta.frame_meta_list
    while l_frame is not None:
        try:
            # Note that l_frame.data needs a cast to pyds.NvDsFrameMeta
            frame_meta = pyds.NvDsFrameMeta.cast(l_frame.data)
        except StopIteration:
            break
        
        # Get frame number , number of rectables to draw and object metadata
        frame_number=frame_meta.frame_num
        num_rects = frame_meta.num_obj_meta
        l_obj=frame_meta.obj_meta_list
        
        while l_obj is not None:
            try:
                # Casting l_obj.data to pyds.NvDsObjectMeta
                obj_meta=pyds.NvDsObjectMeta.cast(l_obj.data)
            except StopIteration:
                break
            # Increment Object class by 1 and Set Box border to Red color     
            obj_counter[obj_meta.class_id] += 1
            obj_meta.rect_params.border_color.set(0.0, 0.0, 1.0, 0.0)
            try: 
                l_obj=l_obj.next
            except StopIteration:
                break
        ################## Setting Metadata Display configruation ############### 
        # Acquiring a display meta object.
        display_meta=pyds.nvds_acquire_display_meta_from_pool(batch_meta)
        display_meta.num_labels = 1
        py_nvosd_text_params = display_meta.text_params[0]
        # Setting display text to be shown on screen
        py_nvosd_text_params.display_text = "Frame Number={} Number of Objects={} Vehicle_count={} Person_count={}".format(frame_number, num_rects, obj_counter[PGIE_CLASS_ID_VEHICLE], obj_counter[PGIE_CLASS_ID_PERSON])
        # Now set the offsets where the string should appear
        py_nvosd_text_params.x_offset = 10
        py_nvosd_text_params.y_offset = 12
        # Font , font-color and font-size
        py_nvosd_text_params.font_params.font_name = "Serif"
        py_nvosd_text_params.font_params.font_size = 10
        # Set(red, green, blue, alpha); Set to White
        py_nvosd_text_params.font_params.font_color.set(1.0, 1.0, 1.0, 1.0)
        # Text background color
        py_nvosd_text_params.set_bg_clr = 1
        # Set(red, green, blue, alpha); set to Black
        py_nvosd_text_params.text_bg_clr.set(0.0, 0.0, 0.0, 1.0)
        # Using pyds.get_string() to get display_text as string to print in notebook
        print(pyds.get_string(py_nvosd_text_params.display_text))
        pyds.nvds_add_display_meta_to_frame(frame_meta, display_meta)
        
        ############################################################################
        
        try:
            l_frame=l_frame.next
        except StopIteration:
            break
    return Gst.PadProbeReturn.OK

In [ ]:
# Lets add probe to get informed of the meta data generated, we add probe to the sink pad  
# of the osd element, since by that time, the buffer would have had got all the metadata.

osdsinkpad = nvosd.get_static_pad("sink")
if not osdsinkpad:
    sys.stderr.write(" Unable to get sink pad of nvosd \n")
    
osdsinkpad.add_probe(Gst.PadProbeType.BUFFER, osd_sink_pad_buffer_probe, 0)

Now with everything defined , we can start the playback and listen the events.

In [ ]:
# start play back and listen to events
print("Starting pipeline \n")
start_time = time.time()
pipeline.set_state(Gst.State.PLAYING)
try:
    loop.run()
except:
    pass
# cleanup
pipeline.set_state(Gst.State.NULL)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Convert video profile to be compatible with Jupyter notebook
!ffmpeg -loglevel panic -y -an -i ../source_code/N2/ds_out.mp4 -vcodec libx264 -pix_fmt yuv420p -profile:v baseline -level 3 ../source_code/N2/output.mp4

In [ ]:
# Display the Output
from IPython.display import HTML
HTML("""
 <video width="640" height="480" controls>
 <source src="../source_code/N2/output.mp4"
 </video>
""".format())

In the next notebook , we will build an multi-stream pipeline performing 4-class object detection.

## Licensing
  
This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).

[Previous Notebook](Getting_started_with_Deepstream_Pipeline.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;
[1](Introduction_to_Deepstream_and_Gstreamer.ipynb)
[2](Getting_started_with_Deepstream_Pipeline.ipynb)
[3]
[4](Multi-stream_pipeline.ipynb)
[5](Multi-stream_Multi_DNN.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[Next Notebook](Multi-stream_pipeline.ipynb)

&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&ensp;
[Home Page](Start_Here.ipynb)